# Mahalanobis outlier detector deployment

Wrap a Mahalanobis anomaly detection model for use as a prediction microservice in seldon-core and deploy on seldon-core running on minikube or a Kubernetes cluster using GCP.

## Dependencies

- [helm](https://github.com/helm/helm)
- [minikube](https://github.com/kubernetes/minikube)
- [s2i](https://github.com/openshift/source-to-image) >= 1.1.13

python packages:
- scikit-learn: pip install scikit-learn --> 0.20.1

## Task

The outlier detector needs to detect computer network intrusions using TCP dump data for a local-area network (LAN) simulating a typical U.S. Air Force LAN. A connection is a sequence of TCP packets starting and ending at some well defined times, between which data flows to and from a source IP address to a target IP address under some well defined protocol. Each connection is labeled as either normal, or as an attack. 

There are 4 types of attacks in the dataset:
- DOS: denial-of-service, e.g. syn flood;
- R2L: unauthorized access from a remote machine, e.g. guessing password;
- U2R:  unauthorized access to local superuser (root) privileges;
- probing: surveillance and other probing, e.g., port scanning.
    
The dataset contains about 5 million connection records.

There are 3 types of features:
- basic features of individual connections, e.g. duration of connection
- content features within a connection, e.g. number of failed log in attempts
- traffic features within a 2 second window, e.g. number of connections to the same host as the current connection

The outlier detector is only using the continuous (18 out of 41) features.

## Test using Kubernetes cluster on GCP or Minikube

Pick Kubernetes cluster on GCP or Minikube.

In [ ]:
minikube = True

In [ ]:
if minikube:
    !minikube start --memory 4096 --feature-gates=CustomResourceValidation=true \
    --extra-config=apiserver.Authorization.Mode=RBAC
else:
    !gcloud container clusters get-credentials standard-cluster-1 --zone europe-west1-b --project seldon-demos

Create a cluster-wide cluster-admin role assigned to a service account named “default” in the namespace “kube-system”.

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin \
--serviceaccount=kube-system:default

In [ ]:
!kubectl create namespace seldon

Add current context details to the configuration file in the seldon namespace.

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Create tiller service account and give it a cluster-wide cluster-admin role.

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

Check deployment rollout status and deploy seldon/spartakus helm charts.

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd \
    --set usage_metrics.enabled=true

In [ ]:
!helm install ../../../helm-charts/seldon-core --name seldon-core \
        --namespace seldon \
        --set ambassador.enabled=true

Check deployment rollout status for seldon core.

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager -n seldon
!kubectl rollout status deploy/seldon-core-seldon-apiserver -n seldon

If Minikube used: create docker image for outlier detector inside Minikube using s2i.

In [ ]:
if minikube:
    !eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3:0.4 seldonio/outlier-mahalanobis:0.1

Install outlier detector helm charts and set *threshold*, *n_components*, *n_stdev* and *start_clip* hyperparameter values.

In [ ]:
!helm install ../../../helm-charts/seldon-od-md \
    --set model.image.name=seldonio/outlier-mahalanobis:0.1 \
    --set model.threshold=25 \
    --set model.n_components=3 \
    --set model.n_stdev=3 \
    --set model.start_clip=50 \
    --name outlier-detector --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --namespace=seldon

## Port forward Ambassador

Run command in terminal:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Import rest requests, load data and test requests

In [ ]:
from utils import get_payload, rest_request_ambassador, send_feedback_rest, get_kdd_data, generate_batch

data = get_kdd_data(percent10=True) # load dataset
print(data.shape)

Generate a random batch from the data

In [ ]:
import numpy as np

samples = 1
fraction_outlier = 0.
X, labels = generate_batch(data,samples,fraction_outlier)
print(X.shape)
print(labels.shape)

Test the rest requests with the generated data. It is important that the order of requests is respected. First we make predictions, then we get the "true" labels back using the feedback request. If we do not respect the order and eg keep making predictions without getting the feedback for each prediction, there will be a mismatch between the predicted and "true" labels. This will result in errors in the produced metrics.

In [ ]:
request = get_payload(X)

In [ ]:
response = rest_request_ambassador("outlier-detector",request,endpoint="localhost:8003")

In [ ]:
send_feedback_rest("outlier-detector",request,response,0,labels,endpoint="localhost:8003")

## Analytics

Install the helm charts for prometheus and the grafana dashboard

In [ ]:
!helm install ../../../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --namespace seldon

## Port forward Grafana dashboard

Run command in terminal:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
```

You can then view an analytics dashboard inside the cluster at http://localhost:3000/dashboard/db/prediction-analytics?refresh=5s&orgId=1. Your IP address may be different. get it via minikube ip. Login with:

Username : admin

password : password (as set when starting seldon-core-analytics above)

Import the outlier-detector-md dashboard from ../../../helm-charts/seldon-core-analytics/files/grafana/configs.

## Run simulation

- Sample random network intrusion data with a certain outlier probability.
- Get payload for the observation.
- Make a prediction.
- Send the "true" label with the feedback.

It is important that the prediction-feedback order is maintained. Otherwise there will be a mismatch between the predicted and "true" labels.

View the progress on the grafana "Outlier Detection" dashboard.

In [ ]:
import time
n_requests = 100
samples = 1
for i in range(n_requests):
    fraction_outlier = .1
    X, labels = generate_batch(data,samples,fraction_outlier)
    request = get_payload(X)
    response = rest_request_ambassador("outlier-detector",request,endpoint="localhost:8003")
    send_feedback_rest("outlier-detector",request,response,0,labels,endpoint="localhost:8003")
    time.sleep(1)

In [ ]:
if minikube:
    !minikube delete